In [1]:
import numpy as np
import torch
import torch.utils.data as Data
import torch.nn as nn
import h5py
import matplotlib.pyplot as plt
from torch.utils.data.sampler import SubsetRandomSampler
import pandas as pd
import copy
from torch.optim.lr_scheduler import ReduceLROnPlateau

In [2]:
#loading data
f = h5py.File("../modulation-prediction/data.hdf5", 'r')
X_train = np.array(f['train'])
X_train = torch.from_numpy(X_train).reshape(X_train.shape[0],1,X_train.shape[2], X_train.shape[1])

X_test = np.array(f['test'])
X_test = torch.from_numpy(X_test).reshape(X_test.shape[0],1,X_test.shape[2], X_test.shape[1])

Y_train = pd.read_csv('../modulation-prediction/train_labels.csv').to_numpy()

print(X_train.shape)
print(Y_train.shape)

torch.Size([30000, 1, 2, 1024])
(30000, 2)


In [ ]:
class_index = {
 'FM': 0,
 'OQPSK':1,
 'BPSK':2,
 '8PSK':3,
 'AM-SSB-SC':4,
 '4ASK':5,
 '16PSK':6,
 'AM-DSB-SC':7, 
 'QPSK': 8, 
 'OOK':9,
}

index_class = {
 0:'FM',
 1:'OQPSK',
 2:'BPSK',
 3:'8PSK',
 4:'AM-SSB-SC',
 5:'4ASK',
 6:'16PSK',
 7:'AM-DSB-SC', 
 8:'QPSK', 
 9:'OOK',
}

classnum = 10
def classToIndex(catg):
    return class_index[catg]

#direct mapping, not one-hot
def setToNum(dataset):
    tensor = torch.zeros(dataset.shape[0], dtype = torch.long)
    for li, catg in enumerate(dataset[:,1]):
        tensor[li] = classToIndex(catg)
    return tensor

# class to name
def classToName(cls):
    return index_class[cls]

In [ ]:
# Hyper-parameters
# sequence_length = 65
# input_size = 2
# hidden_size = 128
num_epochs = 1000
# num_layers = 1
num_classes = 10
batch_size = 100
learning_rate = 0.001

Y_number = setToNum(Y_train)
print(Y_number.shape)
print(Y_number.dtype)

# divide it into batch
train_dataset = Data.TensorDataset(X_train,Y_number)
validation_split = 0.2

dataset_len = len(train_dataset)
indices = list(range(dataset_len))

# Randomly splitting indices:
val_len = int(np.floor(validation_split * dataset_len))
validation_idx = np.random.choice(indices, size=val_len, replace=False)
train_idx = list(set(indices) - set(validation_idx))

## Defining the samplers for each phase based on the random indices:
train_sampler = SubsetRandomSampler(train_idx)
validation_sampler = SubsetRandomSampler(validation_idx)

train_loader = Data.DataLoader(dataset=train_dataset,
                              batch_size=batch_size,
                              sampler = train_sampler,
                              shuffle=False)
test_loader = Data.DataLoader(dataset = X_test,
                              batch_size=batch_size,
                              shuffle=False)
validation_loader = Data.DataLoader(train_dataset, 
                                    batch_size=batch_size,
                                    sampler=validation_sampler)
data_loaders = {"train": train_loader, "val": validation_loader}
data_lengths = {"train": len(train_idx), "val": val_len}


In [ ]:
# 1D Convolutional neural network 
class ConvNet(nn.Module):
    def __init__(self, num_classes=10):
        super(ConvNet, self).__init__()
        
        self.layer1 = nn.Sequential(
            nn.Conv1d(2, 64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2, stride=2, padding = 0))
        self.layer2 = nn.Sequential(
            nn.Conv1d(64,64, kernel_size=3, stride=1, padding=1,),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2, stride=2, padding = 0))
        self.layer3 = nn.Sequential(
            nn.Conv1d(64, 64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2, stride=2, padding = 0))
        self.layer4 = nn.Sequential(
            nn.Conv1d(64, 64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2, stride=2, padding = 0))
        self.layer5 = nn.Sequential(
            nn.Conv1d(64, 64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2, stride=2, padding = 0))
        self.layer6 = nn.Sequential(
            nn.Conv1d(64, 64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2, stride=2, padding = 0))
        self.layer7 = nn.Sequential(
            nn.Conv1d(64, 64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2, stride=2, padding = 0))
        
        self.fc1 = nn.SELU()
        self.fc2 = nn.SELU()
        self.dropout = nn.AlphaDropout(p = 0.3)
        self.fc1 = nn.Linear(64*8, 128)
        self.fc3 = nn.Linear(128, num_classes)
        
    def forward(self, x):
        
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.layer5(out)
        out = self.layer6(out)
        out = self.layer7(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc1(out)
        out = self.fc2(out)
        out = self.dropout(out)
        out = self.fc3(out)
        return out

model = ConvNet(num_classes)

In [ ]:
# 2D Convolutional neural network
class ConvNet(nn.Module):
    def __init__(self, num_classes=10):
        super(ConvNet, self).__init__()
        
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2, padding = 1))
        self.layer2 = nn.Sequential(
            nn.Conv2d(16,32, kernel_size=3, stride=1, padding=1,),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2, padding = 1))
        self.layer3 = nn.Sequential(
            nn.Conv2d(32, 32, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2, padding = 1))
        self.layer4 = nn.Sequential(
            nn.Conv2d(32, 32, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2, padding = 1))
#         self.layer5 = nn.Sequential(
#             nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1),
#             nn.BatchNorm2d(64),
#             nn.ReLU(),
#             nn.MaxPool2d(kernel_size=2, stride=2, padding = 1))
#         self.layer6 = nn.Sequential(
#             nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1),
#             nn.BatchNorm2d(64),
#             nn.ReLU(),
#             nn.MaxPool2d(kernel_size=2, stride=2, padding = 1))
#         self.layer7 = nn.Sequential(
#             nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1),
#             nn.BatchNorm2d(64),
#             nn.ReLU(),
#             nn.MaxPool2d(kernel_size=2, stride=2, padding = 1))
        
#         self.fc1 = nn.SELU()
#         self.fc2 = nn.SELU()
#         self.dropout = nn.AlphaDropout(p = 0.3)
#         self.fc1 = nn.Linear(9*2*64, 128)
        self.fc3 = nn.Linear(65*2*32, num_classes)
        
    def forward(self, x):
        
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
#         out = self.layer5(out)
#         out = self.layer6(out)
#         out = self.layer7(out)
        out = out.reshape(out.size(0), -1)
#         out = self.fc1(out)
#         out = self.fc2(out)
#         out = self.dropout(out)
        out = self.fc3(out)
        return out

model = ConvNet(num_classes)
        


In [ ]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay = 0.006)
# scheduler = ReduceLROnPlateau(optimizer, 'min', factor = 0.5, patience=5)

# Train the model
# total_step = len(train_loader)
bestmodel = 0
best = 0
for epoch in range(num_epochs):
    # Each epoch has a training and validation phase
    for phase in ['train', 'val']:
        if phase == 'train':
            model.train(True)  # Set model to training mode
        else:
            model.train(False)  # Set model to evaluate mode
        correct = 0
        traincorrect = 0
        total = val_len 
        total_step = len(data_loaders[phase])
        for i, (samples, labels) in enumerate(data_loaders[phase]):
        #       print(samples.shape)
            samples = samples
            labels = labels
        #       print(labels.shape)

            # Forward pass
            outputs = model(samples)
            loss = criterion(outputs, labels)
                
            # backward + optimize only if in training phase
            if phase == 'train':
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                
                if (i+1) % 240 == 0:
                    epoch_loss = 0
                    model.eval()
                    for trainsamples, trainlabels in train_loader:
                        trainsamples = trainsamples
                        trainlabels = trainlabels

                        trainoutputs = model(trainsamples)
                        epoch_loss += criterion(trainoutputs, trainlabels)
                        _, trainpredicted = torch.max(trainoutputs.data, 1)
                        traincorrect += (trainpredicted == trainlabels).sum().item()
#                         scheduler.step(epoch_loss)
                    print ('Epoch [{}/{}], Step [{}/{}], Train Loss: {:.4f}, Train Acc: {:.4f}%' 
                        .format(epoch+1, num_epochs, i+1, total_step, loss.item(), 100*traincorrect/24000))

            else:
                _, valpredicted = torch.max(outputs.data, 1)
                correct += (valpredicted == labels).sum().item()
#                 scheduler.step(correct/total)
                if (i+1) % 60 == 0:
                    if(correct / total > best):
                        best = correct / total
                        bestmodel = copy.deepcopy(model) 
                    print ('Epoch [{}/{}], Validation Acc: {:.4f}%, Best Acc: {:.4f}%' 
                        .format(epoch+1, num_epochs, 100 *correct / total, best*100)) 

In [ ]:
# Train accuracy
model = bestmodel
model.eval()
with torch.no_grad():
    traincorrect = 0
    total = 30000
    
    for trainsamples, labels in train_loader:
        trainsamples = trainsamples
        labels = labels
        
        trainoutputs = model(trainsamples)
        _, trainpredicted = torch.max(trainoutputs.data, 1)
        traincorrect += (trainpredicted == labels).sum().item()
        
    print('Train Accuracy of the model over the 30000 train modulations: {} %'.format(100 * traincorrect / total)) 

In [ ]:
# Test the model
model = bestmodel
model.eval()
predictions = np.array([[999]])

# print(predictions)
with torch.no_grad():
    for testsamples in test_loader:
        testsamples = testsamples
        
        outputs = model(testsamples)
        _, predicted = torch.max(outputs.data, 1)
        predicted = predicted.cpu().numpy().reshape(testsamples.shape[0],1)
        predictions = np.vstack((predictions, predicted))
# print(predictions)

In [ ]:
predictionlist = predictions.tolist()
names = []
# print(len(predictionlist))
for i in range(1,len(predictionlist)):
    names.append(classToName(predictionlist[i][0]))
    
# print(names)
outputs = np.asarray(names, dtype = str)
# print(outputs)
categories = pd.DataFrame(outputs, columns = ['Category'])       
categories.to_csv('categories.csv')